In [1]:
import math
from collections import Counter
from matplotlib import pyplot as plt
import random
import pandas as pd
import os
import sys
import csv
import dateutil.parser
from packages.Statistics import correlation
from collections import Counter
import numpy as np
from collections import Counter, defaultdict
from sklearn import linear_model

In [2]:
data = []
data = pd.read_csv("datasets/Car_model_price_modified.csv")

In [3]:
data['Price Drop'] = ''
data = data.drop(columns = ['City','State','Vin','Unnamed: 0'])

In [4]:
data[data == np.inf] = np.nan
data['New Car Price'] = data['New Car Price'].replace(0, np.nan)
data.dropna()


for i in range(len(data)):
    data.loc[i,'Price Drop'] = data['New Car Price'][i]-data['Price'][i]

  
data['Price Drop'] = data['Price Drop'].replace(0, np.nan)
data['Price Drop'] = data['Price Drop'].replace(np.inf, np.nan)
data['Price Drop'] = data['Price Drop'].replace('nan', np.nan)
data.dropna(how = 'any', inplace = True)
data.index = range(len(data))

In [5]:
data.head()

,Price,Year,Mileage,Make,Model,General Model,New Car Price,Price Drop
0,21747,2016,20038,Acura,ILXAutomatic,ILX,25900.0,4153.0
1,13995,2014,67261,Acura,ILX5-Speed,ILX,25900.0,11905.0
2,14999,2015,42989,Acura,ILX5-Speed,ILX,25900.0,10901.0
3,19000,2015,23779,Acura,ILX5-Speed,ILX,25900.0,6900.0
4,21997,2016,21206,Acura,ILXAutomatic,ILX,25900.0,3903.0


In [6]:
reg = linear_model.LinearRegression()

In [7]:
r = 0
coefficient_list = defaultdict(list)
for i in range(len(data)-1):
    
    if data['General Model'][i] != data['General Model'][i+1] or i == len(data)-2:
        
        reg.fit(data[r:i][['Mileage','Year']],data[r:i]['Price Drop'])
        coefficient_list[data['Make'][i]].append([data['General Model'][i],reg.coef_,reg.intercept_,data['New Car Price'][i]])
        r = i+1

In [8]:
car_brand_list = []
for i,j in list(enumerate(Counter(data['Make']),1)):
    car_brand_list.append(j)

In [9]:
k = coefficient_list['Acura'][0][1][0]* 2
print(k)

0.09662940586437639


In [10]:
def price_prediction(make,model,mileage,year):
    for i in range(len(coefficient_list[make])):
        if coefficient_list[make][i][0] == model:
            predicted_drop = coefficient_list[make][i][1][0]*mileage + coefficient_list[make][i][1][1]*year + coefficient_list[make][i][2]
            predicted_price = round(coefficient_list[make][i][3] - predicted_drop)
            print(f"{model} from {make} will approximately cost ${predicted_price}")
            break
        else:
            print('No such model has been found')

In [11]:
price_prediction('Acura','ILX',20038,2016)

ILX from Acura will approximately cost $21290.0
